In [256]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder , OrdinalEncoder , MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer

In [257]:
df = pd.read_csv(r"C:\Users\alisa\OneDrive\Desktop\Machine Learning Note\Pandas Profiling\train (1).csv")
df.sample(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
552,553,0,3,"O'Brien, Mr. Timothy",male,NaN,0,0,330979,7.8292,NaN,Q
57,58,0,3,"Novel, Mr. Mansouer",male,28.5,0,0,2697,7.2292,NaN,C
525,526,0,3,"Farrell, Mr. James",male,40.5,0,0,367232,7.7500,NaN,Q


In [258]:
df.drop(columns=['PassengerId' , 'Name' , 'Ticket' , 'Cabin'] , inplace=True)

In [259]:
df.sample(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
864,0,2,male,24.0,0,0,13.0000,S
223,0,3,male,NaN,0,0,7.8958,S
878,0,3,male,NaN,0,0,7.8958,S


In [260]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

Age me null value hai to me 'mean' ka use karunga or embark me 'most_frequent' ka

In [261]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                 df['Survived'],
                                                 test_size=0.2,
                                                random_state=42)

In [262]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [263]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Define the ColumnTransformer
transformer = ColumnTransformer(transformers=[
    ('age_imputer', SimpleImputer(strategy='mean'), ['Age']),  # Impute Age
    ('embarked_imputer', SimpleImputer(strategy='most_frequent'), ['Embarked']),  # Impute Embarked
    ('sex_encoder', OneHotEncoder(sparse_output=False, drop='first'), ['Sex']),  # One-hot encode Sex
    ('embarked_encoder', OneHotEncoder(sparse_output=False, drop='first'), ['Embarked'])  # One-hot encode Embarked
], remainder='passthrough')  # Pass through other columns unchanged

# Apply the transformation
X_train_transformed = transformer.fit_transform(X_train)
X_test_transformed = transformer.transform(X_test)

# Check the output shape
print("Transformed X_train shape:", X_train_transformed.shape)
print("Transformed X_test shape:", X_test_transformed.shape)


Transformed X_train shape: (712, 10)
Transformed X_test shape: (179, 10)


In [264]:
X_train_transformed

array([[45.5, 'S', 1.0, ..., 0.0, 0.0, 28.5],
       [23.0, 'S', 1.0, ..., 0.0, 0.0, 13.0],
       [32.0, 'S', 1.0, ..., 0.0, 0.0, 7.925],
       ...,
       [41.0, 'S', 1.0, ..., 2.0, 0.0, 14.1083],
       [14.0, 'S', 0.0, ..., 1.0, 2.0, 120.0],
       [21.0, 'S', 1.0, ..., 0.0, 1.0, 77.2875]], dtype=object)

In [265]:
clf = DecisionTreeClassifier()
clf.fit(X_train_transformed,y_train)

ValueError: could not convert string to float: 'S'

colsumnstransform mr problem aaya hai ke wo Embarked ko(ValueError: could not convert string to float: 'S') OHE me convert kar nahi paya pata nahi Q may be Pipeline use karke hoga.

so, iused Manually

In [266]:
si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

X_train_age = si_age.fit_transform(X_train[['Age']])
X_train_embarked = si_embarked.fit_transform(X_train[['Embarked']])

X_test_age = si_age.transform(X_test[['Age']])
X_test_embarked = si_embarked.transform(X_test[['Embarked']])

In [267]:
# one hot encoding Sex and Embarked

ohe_sex = OneHotEncoder(sparse_output=False,handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse_output=False,handle_unknown='ignore')

X_train_sex = ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked = ohe_embarked.fit_transform(X_train_embarked)

X_test_sex = ohe_sex.transform(X_test[['Sex']])
X_test_embarked = ohe_embarked.transform(X_test_embarked)

In [268]:
X_train.sample(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
832,3,male,NaN,0,0,7.2292,C
324,3,male,NaN,8,2,69.5500,S


In [269]:
X_train_rem = X_train.drop(columns=['Sex','Age','Embarked'])


In [270]:
X_test_rem = X_test.drop(columns=['Sex','Age','Embarked'])

In [271]:
X_train_transformed = np.concatenate((X_train_rem,X_train_age,X_train_sex,X_train_embarked),axis=1)
X_train_transformed.shape


(712, 10)

In [272]:
X_test_transformed = np.concatenate((X_test_rem,X_test_age,X_test_sex,X_test_embarked),axis=1)
X_test_transformed.shape

(179, 10)

In [273]:
clf = DecisionTreeClassifier()
clf.fit(X_train_transformed,y_train)

DecisionTreeClassifier()

In [274]:
y_pred = clf.predict(X_test_transformed)
y_pred

array([0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1], dtype=int64)

In [275]:
X_test_transformed

array([[3., 1., 1., ..., 1., 0., 0.],
       [2., 0., 0., ..., 0., 0., 1.],
       [3., 0., 0., ..., 0., 0., 1.],
       ...,
       [3., 1., 5., ..., 0., 0., 1.],
       [2., 0., 0., ..., 0., 0., 1.],
       [3., 1., 1., ..., 0., 0., 1.]])

In [276]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7821229050279329

In [278]:
import pickle
pickle.dump(ohe_sex,open(r'C:\Users\alisa\OneDrive\Desktop\Machine Learning Note\ML Pipeline\pickle file/ohe_sex_Without_pipeline.pkl','wb'))
pickle.dump(ohe_embarked,open(r'C:\Users\alisa\OneDrive\Desktop\Machine Learning Note\ML Pipeline\pickle file/ohe_embarked_Without_pipeline.pkl','wb'))
pickle.dump(clf,open(r'C:\Users\alisa\OneDrive\Desktop\Machine Learning Note\ML Pipeline\pickle file/clf_Without_pipeline.pkl','wb'))